<a href="https://colab.research.google.com/github/lrheckel/Bellevue-MSDS/blob/master/groupby%20apply%20nlargest%20boolean%20mask%20working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Main topics covered during today's session:_

This NB (some important pandas functions):

1. **groupby and apply**
2. **nlargest, using sum, and mean functions**
3. **boolean mask**
4. **complex analysis using all of the above**   

## The purpose of this notebook is to work through some pandas functions and concepts that are commonly used in data analysis. The types of analyses that we cover here are ones that you could possibly be asked to recreate in some fashion, before the semester's end.

In [ ]:
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/nba_stats.csv
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/worst_players.csv
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/best_players.csv
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/top_rebs.csv
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_pandas_apply_functions/main/top_mins.csv

In [ ]:
# some modules we will need
import pandas as pd
import numpy as np

We will be using some data from the National Basketball Association's (NBA) statistics API for this exercise. The data is from the 2017-2020 seasons and include the major statistics for players.

We will import the data into a dataframe called nba_stats and take a quick look at the data.

In [ ]:
# load the data file
# bring in the sample output file
nba_stats = pd.read_csv('nba_stats.csv')
nba_stats.head(5)

#### The info() and describe() functions are good to use when first looking at a dataframe.

info() gives us column information, and describe() gives us some statistical measurements of the dataframe.

In [ ]:
nba_stats.info()
nba_stats.describe()

### Before we get started on the functions, let's take a quick look at some of the key data fields that we will be working with, and some fields whose meaning may not be easily discernble from the name.

- `PLAYER_ID` - The unique ID number for each player.
- `SEASON_ID` - The ID number for each season. The combination of PLAYER_ID and SEASON_ID gives us the primary key for the dataframe.

#### Note that there are 2,139 rows in the dataframe. That means we have 2,139 unique player-season combinations.

- `GP` - Games played during the season.
- `MIN` through `PLUS_MINUS` columns- The individual statistics for the player for that season. Whenever we are working with one of the columns, we will define what that column means in the exercise.
- `*_RANK` - The place that the player ranks for the season for that particular statistic. There is a `*_RANK` column corresponding to each statistical column.


We are not going to work with EVERY column in the dataframe, so let's create a dataframe with a subset of columns that we will be interested in.

In [ ]:
nba_stats_2 = nba_stats[['SEASON_ID','PLAYER_NAME','GP','MIN','PTS','REB','BLK','TOV','PLUS_MINUS']]
nba_stats_2.head(5)

### Let's do a quick review of the apply() function

`apply()` is used to apply a function to a data frame or to a series (column of the data frame).

The basic way to use the function is:

out = `dataframe`.apply(`func`)

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [ ]:
# inline help
# help(df.apply)

### Use the apply() function on a single column of the dataframe

Pass a built-in function to apply().

What is the average number of games that a player played in during any season?

In [ ]:
# note the syntax of using the DOUBLE BRACKETS around the column name.
mean_value = nba_stats_2[['GP']].apply(np.mean)
print(mean_value)

nba_stats_2[['GP']].apply(np.mean)

#### We can also use the apply function on multiple columns or the entire dataframe, but to do so, all of the dataframe columns must be able to be operated on by the function we are applying. With this data, we can apply to multiple columns that are INT and FLOAT, but not to the entire dataframe, because we also have OBJECT data types.

What is the average number of games, points scored, and rebounds for a player in a season?

In [ ]:
nba_stats_2[['GP','PTS','REB']].apply(np.mean)

# returns value error of "could not convert string to float"
#nba_stats_2.apply(np.mean)

As you can see, the functions returns a value for each column.

That is to say, the default way of apply( ) dealing with a dataframe is to take a whole column each time and operate on that column with the function passed.

We can change this default setting by specifying the `axis` parameter, in which axis=0 (the default) applies by column and axis=1 applies by row. We will not demonstrate row-based apply with this dataset.

### Let's do a quick review of the groupby() function

A `groupby()` operation involves some combination of splitting the object, applying a function, and combining the results. This can be used to group large amounts of data and compute operations on these groups.

The basic way to use the function is:

out = `dataframe`.groupby(by=columnname).`function`()

For example:

df.groupby(by=["b"]).sum()

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

In [ ]:
# inline help
# help(df.groupby)

Let's see how this works.

In [ ]:
nba_stats_2.groupby(by=["SEASON_ID"]).mean()

So what `groupby()` on its own does is a dataframe-wide grouping of every APPLICABLE column of the passed-in function, using the "by" parameter that we set. Note that the groupby above does not include the OBJECT column of NAME.

However, we very seldom want to group an entire df in our analyses. Instead we want to generally:

1. Return statistical analyses for individual or multiple columns
2. Grouped by multiple dimensions (columns).

### So how do we do that? By using `groupby()` and `apply()` together.

**The syntax for a single column looks like:**

dataframe.groupby('columnname').apply(function)

**The syntax for a multiple columns looks like:**

dataframe.groupby(['columnname1','columnname2']).apply(function)

Remember that using `axis=0` (the default) will apply the given function to each *column* and `axis=1` will apply the given function to each *row*.

While `Series.apply` works on individual values and `DataFrame.apply` works on `Series` objects (rows or columns are instances of `Series`), `groupby.apply` works on `DataFrame` objects. The cell below is applying the `print` function to each `DataFrame` or "group" in the `groupby`!

In [ ]:
nba_stats_2.groupby('SEASON_ID').apply(np.sum, axis=0)

In [ ]:
nba_stats_2.groupby('SEASON_ID').apply(np.mean, axis=0)

Note the difference in behavior between apply() alone, groupby() alone, and groupby.apply() together.

1. `apply()` by itself gives us the function result for the columns/rows IT IS ABLE TO OPERATE ON. As we saw above, if we try to perform a function on an incompatible column/row, it will return an error.

2. `groupby()` by itself ALSO gives us the function result for the columns/rows IT IS ABLE TO OPERATE ON. However, it will simply not include the columns that the function cannot operate on in, the result set.

3. `groupby()` `apply()` together will return the function result for the columns/rows IT IS ABLE TO OPERATE ON, similar to `groupby()` alone. However, the difference is with the PLAYER_NAME column, in that the sum() function "added" the player names by concatenating (using "+") the strings together. Using np.mean() gave us the result with the "nuisance columns" error message.

### So how might we want to use this is in a real (or testing) scenario?

Return a dataframe that summarizes the total minutes, games played, point, rebounds, blocks, and turnovers for each player, over the 4 seasons.

In [ ]:
nba_stats_2.groupby('PLAYER_NAME').apply(np.sum, axis=0)

So what is the problem with this returned dataframe? And how can we fix it?

Two ways of fixing (and either one works fine).

1. Create a new dataframe by setting your (possible) grouping columns as indices. We will use this method for the remainder of the notebook.
2. Create a new dataframe by keeping only the columns necessary for that particular analysis. We leave this method up to the students to work through.

In [ ]:
# Setting the possible grouping columns as indices.
nba_stats_3 = nba_stats_2.set_index(['PLAYER_NAME','SEASON_ID'])

In [ ]:
nba_stats_3.head()

In [ ]:
nba_stats_3.groupby('SEASON_ID').apply(np.sum, axis=0)

In [ ]:
nba_stats_3.groupby('PLAYER_NAME').apply(np.sum, axis=0)

In [ ]:
nba_stats_3.groupby('PLAYER_NAME').apply(np.mean, axis=0)

### Any questions up to this point?

### Now let's use some functions to do a more complex analysis, by player and season. What we are about to go over would be typical for a multi-point exercise on an exam.

#### We will use the nba_stats_3 dataframe for this. Recall that this dataframe has PLAYER_NAME and SEASON_ID set as the indices.

**Requirement**:  

Return a dataframe, top_rebs, containing the player name and season for the top 5 number of rebounds across the 4 seasons. Include the top 5 plus ties. In other works, if there are ties, keep all of the results, even if it results in more than 5 rows being returned. The dataframe should be sorted from most to least, with ties broken by name in reverse alphabetical order. Rebounds are in the column REB.

Use the nba_stats_3 dataframe as the input for this.

The output dataframe should have the following columns:  `player`, `season`, `total_rebounds`.

To meet this requirement, we will want to use the pandas function:  nlargest.

We have explicitly stated what function to use here, but on an exam, you might see something like "the pandas function nlargest might be useful for this exercise".

The requirement on ties is satisfied by the parameter "keep", and the value of "all".

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nlargest.html

In [ ]:
# top_rebs
### YOUR CODE HERE

Your solution should match the dataframe below.

In [ ]:
top_rebs_soln = pd.read_csv('top_rebs.csv')
top_rebs_soln

### What are your questions on this exercise?

**Requirement**:  

Return a dataframe, top_mins, containing the player name for the top 10 average number of minutes for the 4 seasons together. This means that we want to add up the total number of minutes the player has played and divide by the number of seasons, to get the average. Round the average to 1 decimal place, after sorting. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties).

Include the top 10 plus ties. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties). The dataframe should be sorted from most to least, with ties broken by name in reverse alphabetical order. Minutes are in the column MIN.

Use the nba_stats_3 dataframe as the input for this.

The output dataframe should have the following columns:  `player`, `seasons_played`, `avg_minutes`.

In [ ]:
# top_mins
### YOUR CODE HERE

Your solution should match the dataframe below.

In [ ]:
top_mins_soln = pd.read_csv('top_mins.csv')
top_mins_soln

### What are your questions on this exercise?

### Now let's look at Boolean Masks.

#### What is a boolean mask? While boolean masks are typically used with numpy arrays, they can also be applied to pandas dataframes. We will introduce the concept here and later cover how they are used with numpy arrays, which is different from how they are used with pandas.

**A mask is used filter and return only the rows that meet a certain condition.**

With pandas, we can use comparison operator (<, >, >=, <=, ==), the isin() function, or the contains() function for strings.

Vanderplas has an EXCELLENT introduction to masks in his book, focused on numpy. Chapter linked to here:  https://jakevdp.github.io/PythonDataScienceHandbook/02.06-boolean-arrays-and-masks.html

In [ ]:
# mask to filter by comparison
high_minutes = nba_stats_2[nba_stats_2['MIN'] >= 2000]
high_minutes

Note that the index returned is that of the index in the original dataframes. To reset the index to only those rows in the masked dataframe, use reset_index().

In [ ]:
# mask to filter by comparison
high_minutes_idx = nba_stats_2[nba_stats_2['MIN'] >= 2000].reset_index()
high_minutes_idx

In [ ]:
# mask using isin()
season_2017 = nba_stats_2[nba_stats_2['SEASON_ID'].isin([22017,22018])].reset_index()
season_2017

Now let's do a multiple comparison.

In [ ]:
# mask to filter by multiple comparison
# return the players with 2000 or more minutes in the 2017 and 2018 seasons
high_minutes_idx = nba_stats_2[(nba_stats_2['MIN'] >= 2000) & (nba_stats_2['SEASON_ID'].isin([22017,22018]))].reset_index()
high_minutes_idx

In [ ]:
# mask to filter by string comparison
# return all of the players named Anthony
name_anthony = nba_stats_2[nba_stats_2['PLAYER_NAME'].str.contains('Anthony')].reset_index()
name_anthony

**Requirement**:

In the NBA, for a player to lead in any statistical category, he must have played in a minimum number of games. For a full season, that number is 58 games. If you are interested in a full explanation of the requirements, see the link below.

So let's return the scoring leaders, in points per game, for the 2018 season.

Return a dataframe, top_scorers, containing the player name for the top 5 average points per game for the 2018 season, for players who meet the minimum number of games qualification.

Round the average to 1 decimal place, after sorting. The final dataframe indexes should be from 0 to 4 (or higher, if there are ties).

Include the top 5 plus ties. The final dataframe indexes should be from 0 to 4 (or higher, if there are ties). The dataframe should be sorted from most to least, with ties broken by name in reverse alphabetical order.

Points are in the column PTS and games played are in the column GP.

Use the nba_stats_2 dataframe as the input for this.

The output dataframe should have the following columns:  `player`, `games`, `points`, `PPG`.

https://www.nba.com/stats/help/statminimums

In [ ]:
# top_scorers
### YOUR CODE HERE

Your dataframe results should match those at this link:  https://www.espn.com/nba/stats/_/season/2019/seasontype/2

### What are your questions on this exercise?

### Extra Credit, for fun (will not be covered during Skills OH live session)

**Requirement**:

In the NBA, the metric `PLUS_MINUS` provides a single number for the value of a player. The metric is defined as the difference between the number of points the player's team scores, minus the number of points the opposing team scores, during the time that the player is in the game.

A positive number means that, over the course of the season, the player's team scored that many more points than their opponents when he was on the court. Likewise for a negative number, his team scored that many fewer points.

In general, the best players have the highest `PLUS_MINUS`, and the worst player have the lowest `PLUS_MINUS`.

So let's see who the best and worst players were, during the 2020 season.

Return a dataframe, best_players, containing the top 10 players and their `PLUS_MINUS` value. Include the top 10 plus ties. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties). The dataframe should be sorted from most to least, with ties broken by name in reverse alphabetical order.  

Additionally, return a dataframe, worst_players, containing the bottom 10 players and their `PLUS_MINUS` value. Include the bottom 10 plus ties. The final dataframe indexes should be from 0 to 9 (or higher, if there are ties). The dataframe should be sorted from lowest value to highest value, with ties broken by name in reverse alphabetical order.

The output dataframes should have the following columns:  `PLAYER_NAME`, `PLUS_MINUS`. There is no need to rename the columns from their original names in the source dataframe for this exercise.

Use the nba_stats_2 dataframe as the input for this.

The `nsmallest()` function is analogous to `nlargest` for finding the smallest values.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nsmallest.html





In [ ]:
# best players here
### YOUR CODE HERE

Your solution should match the dataframe below.

In [ ]:
best_players_soln = pd.read_csv('best_players.csv')
best_players_soln

In [ ]:
# worst players here
### YOUR CODE HERE

Your solution should match the dataframe below.

In [ ]:
worst_players_soln = pd.read_csv('worst_players.csv')
worst_players_soln